In [81]:
import pandas as pd
import numpy as np
import re
import requests

## 영어가사 한글 번역 - 최종) Rest Api 이용
* 영어 한글 혼합된 경우 번역이 제대로 되지 않는 문제 발생 

In [2]:
data=pd.read_csv('멜론가사크롤링2010to2021.csv')

In [3]:
eng_lyrics=data[data['lyrics'].apply ( lambda x : x.upper()!=x.lower())] #영어가 들어간 문장 추출 

In [4]:
#영어 글자 수 확인용 
글자수=pd.DataFrame()
글자수=글자수.append(eng_lyrics.lyrics.apply(lambda x : len(x)))
글자수=글자수.transpose()
print('총 글자 수는 :',round(글자수.lyrics.sum()),'자 입니다.')

총 글자 수는 : 318354 자 입니다.


# Google Translate  
5000자 까지 번역 가능 -> 우리가 적용하기에는 부족함

In [23]:
from googletrans import Translator #5000자까지 가능
translator= Translator()

trans=pd.DataFrame()
for i in range(num_split): 
    translator= Translator()
    split_df=data[plus*i:plus*(i+1)]
    trans=trans.append(split_df['lyrics'].apply(lambda x : translator.translate(x,dest='ko').text))
    


# Google Cloud Translate
아래의 방법의 경우 50만자 이하까지 무료, 그 이상의 경우 결제가 됨  
다른 방법을 찾자

In [37]:
# 500000자 이상 돌리면 결제 됨
from google.cloud import translate_v2 as translate
#환경변수 설정
GOOGLE_APPLICATION_CREDENTIALS = 'C:/Users/user/Desktop/보아즈/분석 base/감성분석/translateapi-353704-b6b347eb1216.json' #발급받은 api 위치 
client = translate.Client()
client.translate(eng_lyrics['lyrics'][128], target_language='ko')['translatedText']

'내 마음에서 멀리'

# Rest Api 이용

In [30]:
#rest API 이용
import requests
class Translate :
    base_url = "http://translate.googleapis.com/translate_a/single"
    languages=['en','ko']
    
    def __init__(self, target_lang='en',encoding='utf-8'):
        self.target_lang = target_lang
        self.params = {'client':'gtx', 'sl':'auto' , 'dt':'t','ie':encoding}
        
    def request(self,content):
        res=requests.get(url=Translate.base_url, params={**self.params, 'tl': self.target_lang, 'q':content})
        res.raise_for_status()
        return "".join([caption[0] for caption in list(res.json())[0]])
    
t=Translate('ko')
    
# while True :
#     print(t.request(input()))

In [15]:
#Rest Api 사용 방법
t.request(eng_lyrics['lyrics'][0])

'넌 날 몰라'

번역 진행상황을 확인하기 위하여 반복문으로 돌려줌

In [41]:
all_df_year=pd.DataFrame()
for year in range(2010,2022) :
    df_year=eng_lyrics[eng_lyrics.Year==year] 

    df_year['trans_lyrics']=df_year['lyrics'].apply(lambda x : t.request(x))
    all_df_year=all_df_year.append(df_year)
    print(year,"년도 노래 가사 번역 완료")

all_df_year

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


2010 년도 노래 가사 번역 완료
2011 년도 노래 가사 번역 완료
2012 년도 노래 가사 번역 완료
2013 년도 노래 가사 번역 완료
2014 년도 노래 가사 번역 완료
2015 년도 노래 가사 번역 완료
2016 년도 노래 가사 번역 완료
2017 년도 노래 가사 번역 완료
2018 년도 노래 가사 번역 완료
2019 년도 노래 가사 번역 완료
2020 년도 노래 가사 번역 완료
2021 년도 노래 가사 번역 완료


,lyrics,artist,song_name,Year,trans_lyrics
0,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
1,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
2,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
3,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
4,So shut off boy,미쓰에이,Bad Girl Good Girl,2010,그러니 꺼져 소년
...,...,...,...,...,...
67267,for the happy ending,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,해피엔딩을 위해
67271,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다
67272,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다
67273,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다


In [42]:
# all_df_year.to_csv('가사번역_영어가사만.csv',index=True)

기존 데이터프레임에 trans_lyrics 합쳐주기

In [76]:
 trans_df=pd.merge(data, all_df_year['trans_lyrics'], left_index=True, right_index=True,  how='left') ;trans_df

,lyrics,artist,song_name,Year,trans_lyrics
0,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
1,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
2,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
3,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
4,So shut off boy,미쓰에이,Bad Girl Good Girl,2010,그러니 꺼져 소년
...,...,...,...,...,...
67270,끝이 아니길,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,NaN
67271,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다
67272,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다
67273,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다


nan 값은 기존의 가사로 채워주기

In [77]:
trans_df.trans_lyrics[trans_df['trans_lyrics'].isna()]=trans_df.lyrics
trans_df

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,lyrics,artist,song_name,Year,trans_lyrics
0,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
1,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
2,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
3,You don't know me,미쓰에이,Bad Girl Good Girl,2010,넌 날 몰라
4,So shut off boy,미쓰에이,Bad Girl Good Girl,2010,그러니 꺼져 소년
...,...,...,...,...,...
67270,끝이 아니길,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,끝이 아니길
67271,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다
67272,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다
67273,I wish I make it till the last day,릴보이 (lIlBOI),"CREDIT (Feat. 염따, 기리보이, Zion.T)",2021,마지막날까지 했으면 좋겠다


In [78]:
# trans_df.to_csv('멜론2010_2021_가사번역포함.csv',index=False)

# trans_lyrics 전처리  
  
문제 ) '겉으론 Bad girl' 처럼 영어 + 한글은 번역이 안됨    
### 해결방법  
    1. 영어가 들어간 문장을 삭제  => 선택
    2. 영어만 삭제 ( ex) '겉으론 Bad girl' => '겉으론' ) => 문맥의 흐름이 이상해질 것이라 판단

In [112]:
#해결방법1
final_data=trans_df[trans_df.lyrics.apply(lambda x : x.upper()==x.lower())]
#해결방법2
#trans_df.trans_lyrics.apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣+]','',x)).head(50)

In [113]:
final_data.drop(index=3004,inplace=True )#이상한 값 제거
final_data.to_csv('최종멜론_2010to2021_노래가사크롤링.csv')